In [1]:
import yaml
import platform
import pandas as pd
import glob
from PIL import Image

In [2]:
def check_os():
    os = platform.system()

    if os == 'Darwin':
        return "MacOS"
    elif os == 'Linux':
        return "Linux"
    else:
        return "Unknown OS"
    
operating_system = check_os()


if operating_system == "MacOS":
    root_path = "/Users/johnny/Projects/"
elif operating_system == "Linux":
    root_path = "/home/johnny/Projects/"

In [ ]:
from ultralytics import YOLO
import os
import glob
import pandas as pd
from PIL import Image

# Suponiendo que 'root_path' está definido
images_directory = root_path + 'datasets/custom_dataset_v2/images/val/'
labels_directory = root_path + 'datasets/custom_dataset_v2/labels/val/'

image_files = glob.glob(images_directory + '*.jpg')
model = YOLO('../inference_tools/Evaluation/models/detector_best.pt', task='detect')

df_rows = []

def get_image_resolution(image_path):
    with Image.open(image_path) as img:
        return img.size

for image_file in image_files:
    image_resolution = get_image_resolution(image_file)
    base_name = os.path.basename(image_file).replace('.jpg', '')
    label_file = os.path.join(labels_directory, base_name + '.txt')
    
    if os.path.exists(label_file):
        with open(label_file, 'r') as file:
            annotation_data = file.readlines()
        
        for line in annotation_data:
            class_id, x_center, y_center, width, height = line.strip().split()
            object_width = int(float(width) * image_resolution[0])
            object_height = int(float(height) * image_resolution[1])

            df_rows.append({
                'file_name': base_name + '.jpg',
                'class_id': int(class_id),
                'x_center': float(x_center),
                'y_center': float(y_center),
                'width': float(width),
                'height': float(height),
                'res_width': image_resolution[0],
                'res_height': image_resolution[1],
                'obj_width': object_width,
                'obj_height': object_height,
                'image_path': image_file,
                'label_path': label_file,
            })

df_annotations = pd.DataFrame(df_rows)

def calculate_area(width, height):
    return width * height

def predict_yolov8(image_path, label_path=None):
    results = model(image_path, size=640)
    
    predictions = []  
    metrics = {'recall': 0.0, 'map': 0.0}  
    return predictions, metrics

# Procesamiento adicional para obtener las métricas y predicciones
for index, row in df_annotations.iterrows():
    predictions, metrics = predict_yolov8(row.image_path)

    df_annotations.at[index, 'num_of_predicted_objects'] = len(predictions)
    df_annotations.at[index, 'recall'] = metrics['recall']
    df_annotations.at[index, 'map'] = metrics['map']

    areas = [calculate_area(obj.obj_width, obj.obj_height) for obj in df_annotations.itertuples() if obj.image_path == row.image_path]
    if areas:
        df_annotations.at[index, 'lowest_area'] = min(areas)
        df_annotations.at[index, 'biggest_area'] = max(areas)

df_annotations['num_of_gt_objects'] = df_annotations.groupby('file_name')['file_name'].transform('count')

print(df_annotations.head())

In [ ]:
df_annotations.to_csv('/data-fast/108-data3/ierregue/datasets/custom_dataset_v1/annotations_valid.csv', index=False)

In [ ]:
full_hd_new_data = df_annotations[(df_annotations['res_width'] == 1920) & (df_annotations['res_height'] == 1080)].copy()